In [28]:
import sys
import os
script_dir = os.getcwd() # coleata o diretorio do projeto atual
sys.path.append(script_dir)

In [29]:
import pandas as pd
import numpy as np
import nltk
import random
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras import models, layers
from keras.models import clone_model
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from copy import deepcopy

In [30]:
data = pd.read_csv(r'CSV\tudo_junto_pp.csv')
data.head(5)

,Ano,Enunciado_Alternativas,Area_de_Conhecimento
0,2020,bienalsur una invitación cruz la front ded señ...,Linguagens
1,2020,españ corazón feder dim una gran sorpr hab pre...,Linguagens
2,2020,dispon httpinversorsaludcom acess 18 ago 2017 ...,Linguagens
3,2020,lo orígen habit expresíon che hay alg má argen...,Linguagens
4,2020,habl con acent andaluz telediari la 9 hac uno ...,Linguagens


In [31]:
areas = data['Area_de_Conhecimento'].unique()
areas

array(['Linguagens', 'Ciências Humanas', 'Ciências da Natureza',
       'Matemática'], dtype=object)

In [32]:
# Passo 1: Defina um mapeamento personalizado
mapa_personalizado = {
    'Linguagens': 0,
    'Ciências Humanas': 1,
    'Ciências da Natureza': 2,
    'Matemática': 3
}

# Passo 2: Substitua os valores na coluna usando o mapeamento
data['Area_de_Conhecimento'] = data['Area_de_Conhecimento'].map(mapa_personalizado)

In [33]:
areas = data['Area_de_Conhecimento'].unique()
areas

array([0, 1, 2, 3], dtype=int64)

In [34]:
# Divisão dos dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(data['Enunciado_Alternativas'], data['Area_de_Conhecimento'], test_size=0.2, random_state=1)
X_validation, y_validation = data['Enunciado_Alternativas'], data['Area_de_Conhecimento']

In [35]:
# Vetorização dos textos usando TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(X_train).toarray()
X_test = vectorizer.transform(X_test).toarray()
X_validation = vectorizer.transform(X_validation).toarray()


In [36]:
# Define the MLP Model using PyTorch
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim, dense_layers):
        super(MLP, self).__init__()
        layers = []
        
        # Add the first layer
        layers.append(nn.Linear(input_dim, dense_layers[0]))
        layers.append(nn.ReLU())
        
        # Add hidden layers
        for i in range(1, len(dense_layers)):
            layers.append(nn.Linear(dense_layers[i-1], dense_layers[i]))
            layers.append(nn.ReLU())
        
        # Add output layer
        layers.append(nn.Linear(dense_layers[-1], output_dim))
        layers.append(nn.Softmax(dim=1))  # Softmax for multi-class classification
        
        # Stack layers into a Sequential model
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

# Genetic Algorithm class for optimizing MLP
class GeneticAlgorithm:
    def __init__(self, tamanho_populacao: int, numero_prog: int, alpha: float, MLPdim: tuple, data: tuple) -> None:
        self.tamanho_populacao = tamanho_populacao
        self.numero_prog = numero_prog
        self.alpha = alpha
        self.MLPdim = MLPdim
        self.populacao = []

        # Convert the data into tensors
        self.X_train = torch.tensor(data[0], dtype=torch.float32)
        self.Y_train = torch.tensor(data[1], dtype=torch.long)  # assuming categorical labels
        self.X_test = torch.tensor(data[2], dtype=torch.float32)
        self.Y_test = torch.tensor(data[3], dtype=torch.long)

        self.iniciar_gen()
    
    def iniciar_gen(self) -> None:
        for _ in range(self.tamanho_populacao):
            model = MLP(self.MLPdim[0], self.MLPdim[1], self.MLPdim[2])
            self.populacao.append([model, 0])

    def pontuador(self, resposta_prevista: torch.Tensor, resposta_real: int) -> int:
        return 1 if resposta_prevista == resposta_real else 0
    
    def avaliador(self, individuo) -> int:
        pontuacao = 0
        individuo[0].eval()
        with torch.no_grad():
            resposta_prevista = individuo[0](self.X_train)
            resposta_prevista = torch.argmax(resposta_prevista, dim=1)
        
        for i, pred in enumerate(resposta_prevista):
            pontuacao += self.pontuador(pred.item(), self.Y_train[i].item())
        
        return pontuacao
    
    def avaliador_geracional(self) -> None:
        for index, individuo in enumerate(self.populacao):
            self.populacao[index][1] = self.avaliador(individuo)

    def seleciona_progenitores(self) -> list:
        redes_branch_ordenada = sorted(self.populacao, key=lambda item: item[1], reverse=True)
        return redes_branch_ordenada[:self.numero_prog]

    def gerador_descendentes(self) -> None:
        lista_pais = self.seleciona_progenitores()
        descendentes = self.tamanho_populacao - self.numero_prog
        prox_gen = []

        for x in range(descendentes):
            prox_gen.append(deepcopy(lista_pais[x % len(lista_pais)]))
        for indiv in range(len(prox_gen)):
            prox_gen[indiv][0] = self.mutacao(prox_gen[indiv][0])
        prox_gen.extend(lista_pais)
        self.populacao = prox_gen
    
    def mutacao(self, individuo):
        for param in individuo.parameters():
            if param.requires_grad:
                param.data += self.alpha * torch.randn(param.shape)
        return individuo

    def estrategia_evolutiva(self, n_geracoes: int) -> list:
        for gem in range(n_geracoes):
            self.avaliador_geracional()
            self.gerador_descendentes()
            avaliador = sorted(self.populacao, key=lambda item: item[1], reverse=True)
            print([x[1] for x in avaliador])
            print(f"EPOCA: {gem}/{n_geracoes}")
            
        return self.populacao[:1]


In [37]:
y_train = [x for x in y_train]
y_test = [x for x in y_test]

In [38]:
# self, tamanho_populacao : int, numero_prog : int, alpha : float, MLPdim : tuple[3], data : tuple[4]

tamanho_populacao = 1000
numero_prog = 20
alpha = 10
rede_neural = (X_train.shape[1], len(areas), [64, 32, 64])
data = (X_train, y_train, X_test, y_test)

algoritimo_genetico = GeneticAlgorithm(tamanho_populacao, numero_prog, alpha, rede_neural, data)

In [39]:
algoritimo_genetico.estrategia_evolutiva(100)

[176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173,

In [27]:
import random
import numpy as np
import torch
import torch.nn as nn
from deap import base, creator, tools, algorithms

# Definindo a classe do MLP
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim, dense_layers):
        super(MLP, self).__init__()
        layers = []
        layers.append(nn.Linear(input_dim, dense_layers[0]))
        layers.append(nn.ReLU())
        
        for i in range(1, len(dense_layers)):
            layers.append(nn.Linear(dense_layers[i-1], dense_layers[i]))
            layers.append(nn.ReLU())
        
        layers.append(nn.Linear(dense_layers[-1], output_dim))
        layers.append(nn.Softmax(dim=1))
        
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

# Função para inicializar os pesos da rede
def init_individual(ind_class, input_dim, output_dim, dense_layers):
    model = MLP(input_dim, output_dim, dense_layers)
    weights = []
    for param in model.parameters():
        weights.extend(param.detach().numpy().flatten())  # Extrai os pesos da rede
    return ind_class(weights)

# Função de avaliação
def evaluate(individual, input_dim, output_dim, dense_layers, X_train, Y_train):
    model = MLP(input_dim, output_dim, dense_layers)
    start = 0
    for param in model.parameters():
        end = start + param.numel()
        param.data = torch.tensor(np.array(individual[start:end]).reshape(param.shape), dtype=torch.float32)
        start = end

    # Avaliar a acurácia
    model.eval()
    with torch.no_grad():
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        Y_train_tensor = torch.tensor(Y_train, dtype=torch.long)
        Y_pred = model(X_train_tensor)
        Y_pred = torch.argmax(Y_pred, axis=1)
        accuracy = (Y_pred == Y_train_tensor).sum().item() / len(Y_train)
    
    return accuracy,

class GeneticAlgorithm:
    def __init__(self, tamanho_populacao, numero_prog, alpha, MLPdim, data):
        self.tamanho_populacao = tamanho_populacao
        self.numero_prog = numero_prog
        self.alpha = alpha
        self.MLPdim = MLPdim
        self.X_train = data[0]
        self.Y_train = data[1]
        self.X_test = data[2]
        self.Y_test = data[3]
        
        # Configurando DEAP
        creator.create("FitnessMax", base.Fitness, weights=(1.0,))
        creator.create("Individual", list, fitness=creator.FitnessMax)
        
        self.toolbox = base.Toolbox()
        self.toolbox.register("individual", init_individual, creator.Individual, *self.MLPdim)
        self.toolbox.register("population", tools.initRepeat, list, self.toolbox.individual)
        self.toolbox.register("evaluate", evaluate, input_dim=self.MLPdim[0], output_dim=self.MLPdim[1], dense_layers=self.MLPdim[2], X_train=self.X_train, Y_train=self.Y_train)
        self.toolbox.register("mate", tools.cxTwoPoint)
        self.toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=self.alpha, indpb=0.1)
        self.toolbox.register("select", tools.selTournament, tournsize=3)
        
    def iniciar_gen(self):
        self.population = self.toolbox.population(n=self.tamanho_populacao)
        
    def avaliador_geracional(self):
        for ind in self.population:
            if not ind.fitness.valid:
                ind.fitness.values = self.toolbox.evaluate(ind)
    
    def gerador_descendentes(self):
        offspring = self.toolbox.select(self.population, len(self.population) - self.numero_prog)
        offspring = list(map(self.toolbox.clone, offspring))
        
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < 0.5:
                self.toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values
                
        for mutant in offspring:
            if random.random() < 0.2:
                self.toolbox.mutate(mutant)
                del mutant.fitness.values
        
        self.population[:] = self.toolbox.select(self.population + offspring, len(self.population))
        
    def estrategia_evolutiva(self, n_geracoes):
        self.iniciar_gen()
        for _ in range(n_geracoes):
            self.avaliador_geracional()
            self.gerador_descendentes()
            
            fits = [ind.fitness.values[0] for ind in self.population if ind.fitness.valid]
            if fits:  # Verifica se há valores de fitness válidos
                print(f"Min: {np.min(fits)}")
                print(f"Max: {np.max(fits)}")
                print(f"Avg: {np.mean(fits)}")

        return self.toolbox.select(self.population, 1)[0]

if __name__ == "__main__":
    
    tamanho_populacao = 100
    numero_prog = 20
    alpha = 0.1
    rede_neural = (X_train.shape[1], len(set(y_train)), [64, 32, 64])
    data = (X_train, y_train, X_test, y_test)
    
    algoritmo_genetico = GeneticAlgorithm(tamanho_populacao, numero_prog, alpha, rede_neural, data)
    melhor_individuo = algoritmo_genetico.estrategia_evolutiva(n_geracoes=10)
    
    print("Melhor indivíduo:", melhor_individuo)


c:\Users\Pichau\AppData\Local\Programs\Python\Python311\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\Pichau\AppData\Local\Programs\Python\Python311\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


KeyboardInterrupt: 